In [1]:
import heeps
import numpy as np
from astropy.io import fits

In [2]:
def get_psfs(conf, case='scao_only', modes=['RAVC', 'APPIMG', 'CVC', 'CLC']):
    conf.update(
        mode = 'ELT',
        f_phase = 'wavefront/cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_%s'%case + '_L_285.fits',
        apo_drift = 0.02, # (% ptv)
        f_amp = 'wavefront/talbot/DynamicV08/Talbot_LM_20220223' + '_%s.fits'
            %('_rep_1_field_50' if 'static' in case else 'rep_1_-0h30'),# else rep),
        vc_chrom_leak = 2e-3,
        avg = True,
        hfov = 1.065,
        ngrid = 2048,
    )
    conf = heeps.config.read_config(**conf)
    conf['band_specs']['L']['pscale'] = 5.47/2
    conf = heeps.config.update_config(**conf)
    wf = heeps.pupil.pupil(**conf)
    psf = heeps.wavefront.propagate(wf, **conf)
    elt_peak = np.max(psf)
    print('ELT peak = %.4f'%elt_peak)
    psf /= elt_peak
    fits.writeto(conf['dir_output'] + 'onaxis_PSF_L_ELT.fits', psf, overwrite=True)
    
    verbose = False
    for conf['mode'] in modes:
        if 'RAVC' in conf['mode']:
            conf.update(ls_dRext=0.0477, ls_dRint=0.04, ls_dRspi=0.0249)
        elif 'CVC' in conf['mode']:
            conf.update(ls_dRext=0.0209, ls_dRint=0.09, ls_dRspi=0.0245)
        elif 'CLC' in conf['mode']:
            conf.update(ls_dRext=0.1095, ls_dRint=0.0491, ls_dRspi=0.0317)
        conf = heeps.config.update_config(**conf, verbose=verbose)
        wf = heeps.pupil.pupil(**conf)
        if 'VC' in conf['mode']:
            psf_OFF = heeps.wavefront.propagate(wf, onaxis=False, **conf)
            psf_ON = heeps.wavefront.propagate(wf, onaxis=True, **conf)
            print('mode = %s, leakage = %.2e'%(conf['mode'], np.sum(psf_ON)/np.sum(psf_OFF)))
        else:
            psf_ON = heeps.wavefront.propagate(wf, onaxis=True, **conf)
        psf_ON /= elt_peak
        fits.writeto(conf['dir_output'] + 'onaxis_PSF_L_%s.fits'%conf['mode'], psf_ON, overwrite=True)

### ideal

In [3]:
conf = dict(
    dir_output = 'instant_psf/ideal',
    nframes_avg = 1,
    add_phase = False,
)
get_psfs(conf)

   2022-07-17 06:21:12, e2e simulation using 56 cores
   2022-07-17 06:21:16, completed in 4.97 seconds
ELT peak = 0.0122
   2022-07-17 06:21:20, e2e simulation using 56 cores
   2022-07-17 06:21:26, completed in 5.46 seconds
   2022-07-17 06:21:27, e2e simulation using 56 cores
   2022-07-17 06:21:38, completed in 10.68 seconds
mode = RAVC, leakage = 2.15e-03
   2022-07-17 06:21:40, e2e simulation using 56 cores
   2022-07-17 06:21:45, completed in 5.27 seconds
   2022-07-17 06:21:46, e2e simulation using 56 cores
   2022-07-17 06:21:52, completed in 5.1 seconds
   2022-07-17 06:21:53, e2e simulation using 56 cores
   2022-07-17 06:22:03, completed in 10.45 seconds
mode = CVC, leakage = 2.59e-02
   2022-07-17 06:22:05, e2e simulation using 56 cores
   2022-07-17 06:22:17, completed in 12.14 seconds


### scao only

In [4]:
conf = dict(
    dir_output = 'instant_psf/scao_only',
    nframes_avg = 1,
    add_phase = True,
)
get_psfs(conf)

   2022-07-17 06:22:18, e2e simulation using 56 cores
   2022-07-17 06:22:23, completed in 4.66 seconds
ELT peak = 0.0118
   2022-07-17 06:22:26, e2e simulation using 56 cores
   2022-07-17 06:22:32, completed in 6.01 seconds
   2022-07-17 06:22:35, e2e simulation using 56 cores
   2022-07-17 06:22:45, completed in 10.78 seconds
mode = RAVC, leakage = 2.36e-02
   2022-07-17 06:22:47, e2e simulation using 56 cores
   2022-07-17 06:22:53, completed in 5.47 seconds
   2022-07-17 06:22:55, e2e simulation using 56 cores
   2022-07-17 06:23:00, completed in 4.97 seconds
   2022-07-17 06:23:02, e2e simulation using 56 cores
   2022-07-17 06:23:12, completed in 9.7 seconds
mode = CVC, leakage = 4.71e-02
   2022-07-17 06:23:14, e2e simulation using 56 cores
   2022-07-17 06:23:25, completed in 10.51 seconds


### long exposure

In [5]:
conf = dict(
    dir_output = 'instant_psf/long_exp',
    nframes_avg = 100,
    add_phase = True,
)
get_psfs(conf)

   2022-07-17 06:23:27, e2e simulation using 56 cores
   2022-07-17 06:23:57, completed in 30.04 seconds
ELT peak = 0.0118
   2022-07-17 06:24:01, e2e simulation using 56 cores
   2022-07-17 06:25:03, completed in 62.19 seconds
   2022-07-17 06:25:07, e2e simulation using 56 cores
   2022-07-17 06:26:13, completed in 66.21 seconds
mode = RAVC, leakage = 2.43e-02
   2022-07-17 06:26:16, e2e simulation using 56 cores
   2022-07-17 06:27:08, completed in 52.65 seconds
   2022-07-17 06:27:13, e2e simulation using 56 cores
   2022-07-17 06:28:10, completed in 56.72 seconds
   2022-07-17 06:28:13, e2e simulation using 56 cores
   2022-07-17 06:29:16, completed in 62.93 seconds
mode = CVC, leakage = 4.74e-02
   2022-07-17 06:29:20, e2e simulation using 56 cores
   2022-07-17 06:30:23, completed in 63.1 seconds


In [7]:
conf = dict(
    dir_output = 'instant_psf/long_exp_AE',
    nframes_avg = 100,
    f_pupil = 'pupil/ELT_fullM1_refl_0.02rms.fits',
    add_phase = True,
    add_apo_drift = True,
    add_amp = True,
    add_cl_vort = True,
)
get_psfs(conf, case='all_ncpa', modes=['RAVC'])

   2022-07-17 06:37:21, e2e simulation using 56 cores
   2022-07-17 06:37:55, completed in 33.07 seconds
ELT peak = 0.0114
   2022-07-17 06:38:05, e2e simulation using 56 cores
   2022-07-17 06:39:01, completed in 56.46 seconds
   2022-07-17 06:39:13, e2e simulation using 56 cores
   2022-07-17 06:40:23, completed in 70.47 seconds
mode = RAVC, leakage = 3.54e-02


In [8]:
conf = dict(
    dir_output = 'instant_psf/long_exp_AE_misseg',
    nframes_avg = 100,
    f_pupil = 'pupil/ELT_fullM1_refl_0.02rms_misseg_1x7.fits',
    add_phase = True,
    add_apo_drift = True,
    add_amp = True,
    add_cl_vort = True,
)
get_psfs(conf, case='all_ncpa', modes=['RAVC'])

   2022-07-17 06:40:35, e2e simulation using 56 cores
   2022-07-17 06:41:07, completed in 32.02 seconds
ELT peak = 0.0113
   2022-07-17 06:41:18, e2e simulation using 56 cores
   2022-07-17 06:42:16, completed in 58.01 seconds
   2022-07-17 06:42:27, e2e simulation using 56 cores
   2022-07-17 06:43:47, completed in 79.56 seconds
mode = RAVC, leakage = 4.81e-02
